# 🎨 Valentina High-Quality Dataset Generator - Google Colab

Este notebook gera um dataset de **alta qualidade** para treinar a LoRA facial da Valentina usando a GPU superior do Google Colab.

## 🚀 Vantagens do Colab:
- **40-50 inference steps** vs 8 no MacBook
- **Resolução superior** (1024x1024+)
- **Melhor qualidade** de detalhes faciais
- **Múltiplas variações** profissionais

## 📋 Stack Utilizada:
- **Base**: FLUX.1-dev (máxima qualidade)
- **1º LoRA**: NSFW_MASTER_FLUX (capacidades NSFW)
- **2º LoRA**: Midjourney LoRA (qualidade cinematográfica)
- **GPU**: A100 optimized com 50 inference steps
- **Output**: 6 imagens de referência premium sequenciais

⚠️ **Execute as células em ordem para gerar o dataset otimizado!**

In [ ]:
# 🎯 CONFIGURAÇÕES PARA GERAÇÃO DE ALTA QUALIDADE - FLUX.1-dev + LoRAs Sequenciais

# Modelo base e LoRAs sequenciais
BASE_MODEL_ID = "black-forest-labs/FLUX.1-dev"  # Base model para máxima qualidade
NSFW_LORA_ID = "Keltezaa/NSFW_MASTER_FLUX"       # Primeiro LoRA: NSFW
MIDJOURNEY_LORA_FILENAME = "midjourney_LORA.safetensors"  # Segundo LoRA: Midjourney (local)

# Configurações de qualidade PREMIUM (A100 Optimized)
INFERENCE_STEPS = 50      # Máxima qualidade possível na A100
GUIDANCE_SCALE = 7.5      # Balanço ideal qualidade/criatividade  
RESOLUTION = 1024         # Resolução máxima para detalhes
NSFW_LORA_WEIGHT = 0.8    # Peso do primeiro LoRA (NSFW)
MIDJOURNEY_LORA_WEIGHT = 0.7  # Peso do segundo LoRA (Midjourney)

# Configurações de diversidade
NUM_IMAGES = 6        # 6 variações diversas
SEEDS = [42, 7777, 12345, 98765, 55555, 33333]  # Seeds específicas

# Prompts especializados para dataset
BASE_CHARACTERISTICS = """A stunning 23-year-old woman named Valentina Moreau. Oval face with delicately defined jawline and prominent cheekbones, large almond-shaped deep brown expressive eyes with long dark lashes, perfectly arched eyebrows, full plump lips with pronounced cupid's bow in natural pink tone, straight proportional nose with slightly tapered tip. Flawless sun-kissed golden-olive skin with healthy glow. Rich dark brown hair with subtle chocolate and mahogany highlights, long flowing past shoulders with voluminous natural waves, dense and shiny hair. Curvy and toned silhouette, confident posture."""

# Variações de pose e iluminação para dataset diverso
DATASET_PROMPTS = [
    f"{BASE_CHARACTERISTICS} Professional headshot, direct gaze, neutral expression, studio lighting, white background, hyperrealistic portrait photography, 8k resolution, commercial photography",
    
    f"{BASE_CHARACTERISTICS} Three-quarter angle portrait, subtle smile, natural daylight, window lighting, elegant and sophisticated, professional photography, high detail, photorealistic",
    
    f"{BASE_CHARACTERISTICS} Side profile portrait, looking away, dramatic rim lighting, dark background, artistic portrait, professional studio photography, cinematic lighting, ultra detailed",
    
    f"{BASE_CHARACTERISTICS} Looking up perspective, gentle smile, golden hour lighting, outdoor natural light, warm tones, Instagram-worthy, glamour photography, high quality",
    
    f"{BASE_CHARACTERISTICS} Close-up portrait, intense direct gaze, soft box lighting, professional makeup, fashion photography style, magazine quality, hyperrealistic details",
    
    f"{BASE_CHARACTERISTICS} Full face frontal view, serene expression, balanced lighting, neutral background, reference photo style, documentary photography, crystal clear details"
]

# Paths no Colab
COLAB_MODELS_PATH = "/content/models"
COLAB_OUTPUT_PATH = "/content/valentina_hq_dataset"

print("🎨 Configurações de ALTA QUALIDADE carregadas:")
print(f"📊 Inference Steps: {INFERENCE_STEPS} (vs 8 no MacBook)")
print(f"🎯 Resolução: {RESOLUTION}x{RESOLUTION}")
print(f"🖼️  Imagens a gerar: {NUM_IMAGES}")
print(f"🎭 Variações: poses, ângulos e iluminações diversos")
print(f"⭐ Qualidade: PREMIUM para treinamento superior")

In [ ]:
# 📦 INSTALAÇÃO OTIMIZADA DE DEPENDÊNCIAS VIA UV
print("🔧 Instalando dependências usando uv para maior robustez...")

# 1. Instalar uv
print("⚙️ Instalando uv...")
!pip install -q uv
print("✅ uv instalado.")

# 2. Definir o conteúdo do requirements.txt baseado no pyproject.toml
requirements_content = """\
torch>=2.1.0,<2.4.0
torchvision>=0.16.0,<0.19.0
torchaudio>=2.1.0,<2.4.0
diffusers[torch]>=0.27.0,<1.0
transformers>=4.44.0,<5.0
accelerate>=0.32.0,<1.0
safetensors>=0.4.0,<0.5.0
xformers>=0.0.27,<0.0.29
pillow>=10.0.0,<11.0.0
opencv-python>=4.10.0,<5.0
huggingface-hub>=0.24.5,<1.0
sentencepiece>=0.2.0,<0.3.0
tokenizers>=0.19.0,<0.20.0
protobuf>=5.27.0,<6.0.0
numpy>=2.0.0,<3.0.0
requests>=2.32.0,<3.0.0
scipy>=1.14.0,<2.0.0
matplotlib>=3.9.0,<4.0.0
omegaconf>=2.3.0,<3.0.0
einops>=0.8.0,<0.9.0
invisible-watermark>=0.2.0,<0.3.0
compel>=2.0.0,<3.0.0
wandb>=0.17.0,<0.18.0
peft>=0.12.0,<0.13.0
bitsandbytes>=0.43.0,<0.44.0
gradio>=4.39.0,<5.0.0
albumentations>=1.4.0,<2.0.0
imageio>=2.34.0,<3.0.0
scikit-image>=0.24.0,<0.25.0
tqdm>=4.66.0,<5.0.0
ftfy>=6.2.0,<7.0.0
tensorboard>=2.16.0,<3.0.0
easydict>=1.13.0,<2.0.0
clean-fid==0.1.35
torchmetrics>=1.4.0,<2.0.0
kornia>=0.7.0,<0.8.0
lpips>=0.1.4,<0.2.0
controlnet_aux>=0.0.7,<0.0.8
segment-anything>=1.0.0,<2.0.0
rembg[gpu]>=2.0.56,<3.0.0
moviepy>=1.0.3,<2.0.0
typer>=0.12.0,<0.13.0
rich>=13.7.0,<14.0.0
shellingham>=1.5.0,<2.0.0
"""

# 3. Criar o arquivo requirements.txt no ambiente do Colab
requirements_file_path = "/content/valentina_requirements.txt"
with open(requirements_file_path, "w") as f:
    f.write(requirements_content)
print(f"📄 {requirements_file_path} criado.")

# 4. Instalar dependências usando uv pip install
print(f"🚀 Instalando dependências de {requirements_file_path} com uv...")
print("🕒 Isso pode levar alguns minutos...")
!uv pip install -q -r {requirements_file_path} --extra-index-url https://download.pytorch.org/whl/cu121 --index-strategy unsafe-best-match

print("✅ Dependências instaladas com sucesso usando uv!")
print("🎯 Ambiente otimizado para geração premium")

# Verificação opcional (descomentar para depurar e ver as versões instaladas)
# print("\\nVerificando versões instaladas (amostra):")
# !pip show torch torchvision torchaudio xformers transformers huggingface-hub opencv-python diffusers accelerate bitsandbytes clean-fid

In [ ]:
# 🖥️ DETECÇÃO E OTIMIZAÇÃO DA GPU
import torch
import subprocess

print("🔍 Verificando GPU disponível...")
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader,nounits

# Detectar GPU específica
gpu_name = subprocess.check_output(["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"]).decode().strip()
print(f"\n🚀 GPU detectada: {gpu_name}")

# Otimizar configurações baseado na GPU
if "T4" in gpu_name:
    print("⚡ Configurações Tesla T4 (15GB)")
    DTYPE = torch.float16
    ENABLE_CPU_OFFLOAD = True
    BATCH_SIZE = 1
    INFERENCE_STEPS = 40  # Reduzido para T4
elif "V100" in gpu_name:
    print("💎 Configurações V100 (16GB)")
    DTYPE = torch.float16
    ENABLE_CPU_OFFLOAD = False
    BATCH_SIZE = 1
    INFERENCE_STEPS = 45  # Melhor que T4
elif "A100" in gpu_name:
    print("🔥 Configurações A100 PREMIUM - Máxima Qualidade")
    DTYPE = torch.bfloat16  # Melhor precisão na A100
    ENABLE_CPU_OFFLOAD = False
    BATCH_SIZE = 1  # Foco em qualidade, não velocidade
    INFERENCE_STEPS = 50  # Máxima qualidade possível
    print("🎯 A100 detectada: Configurando para máxima qualidade com LoRAs sequenciais")
elif "H100" in gpu_name:
    print("🚀 Configurações H100 ULTRA PREMIUM")
    DTYPE = torch.bfloat16
    ENABLE_CPU_OFFLOAD = False
    BATCH_SIZE = 2
    INFERENCE_STEPS = 50
else:
    print("📱 Configurações padrão")
    DTYPE = torch.float16
    ENABLE_CPU_OFFLOAD = True
    BATCH_SIZE = 1
    INFERENCE_STEPS = 35

print(f"✅ Configurado: {DTYPE}, CPU offload: {ENABLE_CPU_OFFLOAD}")
print(f"🎯 Steps finais: {INFERENCE_STEPS}")

# Limpeza inicial
torch.cuda.empty_cache()
print(f"🧹 Memória GPU limpa")

# 📦 INSTALAÇÃO OTIMIZADA DE DEPENDÊNCIAS VIA UV
print("🔧 Instalando dependências usando uv para maior robustez...")

# 1. Instalar uv
print("⚙️ Instalando uv...")
!pip install -q uv
print("✅ uv instalado.")

# 2. Upload do arquivo uv.lock
from google.colab import files
print("📤 Faça upload do arquivo uv.lock...")
uploaded = files.upload()

# 3. Sincronizar dependências usando uv.lock com estratégia de índice insegura
print("🚀 Instalando dependências com uv pip sync usando uv.lock...")
!uv pip sync uv.lock --index-strategy unsafe-best-match

print("✅ Dependências instaladas com sucesso usando uv!")
print("🎯 Ambiente otimizado para geração premium")

In [ ]:
# 🔐 AUTENTICAÇÃO HUGGING FACE
from huggingface_hub import notebook_login

print("🔑 Faça login na sua conta Hugging Face para acessar modelos premium:")
notebook_login()

In [ ]:
# 📁 CRIAÇÃO DA ESTRUTURA DE DIRETÓRIOS
import os

directories = [
    COLAB_MODELS_PATH,
    COLAB_OUTPUT_PATH,
    f"{COLAB_OUTPUT_PATH}/metadata"
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"📂 Criado: {directory}")

print("\n✅ Estrutura preparada para dataset premium!")

In [ ]:
# 📤 UPLOAD DA MIDJOURNEY LORA (OPCIONAL - 2º LORA)
from google.colab import files
import shutil

print("🎨 Upload da Midjourney LoRA para qualidade cinematográfica:")
print("\n📝 Arquitetura Sequential LoRA:")
print(f"   1️⃣ Base: {BASE_MODEL_ID}")
print(f"   2️⃣ NSFW LoRA: {NSFW_LORA_ID} (automático)")
print(f"   3️⃣ Midjourney LoRA: {MIDJOURNEY_LORA_FILENAME} (upload opcional)")
print("\n⚠️ Arquivo opcional. Se não fornecido, usará apenas NSFW LoRA.")
print(f"\n📤 Esperado: {MIDJOURNEY_LORA_FILENAME}")

uploaded_files = files.upload()

if MIDJOURNEY_LORA_FILENAME in uploaded_files:
    # Mover para pasta de modelos
    lora_path = f"{COLAB_MODELS_PATH}/{MIDJOURNEY_LORA_FILENAME}"
    shutil.move(MIDJOURNEY_LORA_FILENAME, lora_path)
    
    # Verificar tamanho
    size_mb = os.path.getsize(lora_path) / (1024*1024)
    print(f"✅ Midjourney LoRA carregada: {size_mb:.1f}MB")
    print(f"📁 Localização: {lora_path}")
    
    USE_MIDJOURNEY_LORA = True
    print(f"\n🎯 Configuração Final:")
    print(f"   • NSFW Weight: {NSFW_LORA_WEIGHT}")
    print(f"   • Midjourney Weight: {MIDJOURNEY_LORA_WEIGHT}")
else:
    print("⚠️ Midjourney LoRA não fornecida.")
    print("🚀 Continuando com configuração: Base + NSFW LoRA")
    USE_MIDJOURNEY_LORA = False
    lora_path = None

print(f"\n🎨 Usando Midjourney LoRA: {USE_MIDJOURNEY_LORA}")

In [ ]:
# 🚀 CARREGAMENTO DO PIPELINE OTIMIZADO
from diffusers import FluxPipeline
import torch
import gc

print(f"📦 Carregando modelo base: {BASE_MODEL_ID}")
print("⏱️ Isso pode levar alguns minutos...")

try:
    # Carregar pipeline FLUX.1-dev base
    pipeline = FluxPipeline.from_pretrained(
        BASE_MODEL_ID,
        torch_dtype=DTYPE,
        use_safetensors=True
    )
    
    print("🔧 Configurando pipeline...")
    
    # Otimizações de memória
    if ENABLE_CPU_OFFLOAD:
        pipeline.enable_model_cpu_offload()
        print("💾 CPU offload ativado")
    else:
        pipeline.to("cuda")
        print("🚀 Pipeline totalmente na GPU")
    
    # SEQUENTIAL LORA APPLICATION
    # 1️⃣ Primeiro: Carregar NSFW LoRA do HuggingFace
    print(f"\n🔥 Aplicando NSFW LoRA: {NSFW_LORA_ID}")
    print("📥 Baixando do HuggingFace...")
    pipeline.load_lora_weights(NSFW_LORA_ID, adapter_name="nsfw")
    print(f"✅ NSFW LoRA aplicada com peso {NSFW_LORA_WEIGHT}")
    
    # 2️⃣ Segundo: Carregar Midjourney LoRA se disponível
    lora_adapters = ["nsfw"]
    lora_weights = [NSFW_LORA_WEIGHT]
    
    if USE_MIDJOURNEY_LORA and lora_path:
        print(f"\n🎨 Aplicando Midjourney LoRA: {lora_path}")
        pipeline.load_lora_weights(lora_path, adapter_name="midjourney")
        lora_adapters.append("midjourney")
        lora_weights.append(MIDJOURNEY_LORA_WEIGHT)
        print(f"✅ Midjourney LoRA aplicada com peso {MIDJOURNEY_LORA_WEIGHT}")
    
    # Configurar adaptadores sequenciais
    pipeline.set_adapters(lora_adapters, adapter_weights=lora_weights)
    
    print(f"\n🎯 Pipeline configurado com:")
    print(f"   • Base: {BASE_MODEL_ID}")
    print(f"   • LoRAs: {' + '.join(lora_adapters)}")
    print(f"   • Pesos: {lora_weights}")
    print("\n🚀 Pipeline pronto para geração de máxima qualidade!")
    
except Exception as e:
    print(f"❌ Erro ao carregar pipeline: {e}")
    raise

# Limpeza de memória
gc.collect()
torch.cuda.empty_cache()
print("🧹 Memória otimizada")

In [ ]:
# 🎨 GERAÇÃO DO DATASET DE ALTA QUALIDADE
import time
from PIL import Image
import json

print("🚀 INICIANDO GERAÇÃO DO DATASET PREMIUM")
print("=" * 50)
print(f"📊 Configurações:")
print(f"   • Resolução: {RESOLUTION}x{RESOLUTION}")
print(f"   • Inference Steps: {INFERENCE_STEPS}")
print(f"   • Guidance Scale: {GUIDANCE_SCALE}")
print(f"   • Base Model: {BASE_MODEL_ID}")
print(f"   • NSFW LoRA: {NSFW_LORA_ID} (peso: {NSFW_LORA_WEIGHT})")
if USE_MIDJOURNEY_LORA:
    print(f"   • Midjourney LoRA: {MIDJOURNEY_LORA_FILENAME} (peso: {MIDJOURNEY_LORA_WEIGHT})")
else:
    print(f"   • Midjourney LoRA: Não usado")
print(f"   • Imagens: {NUM_IMAGES}")
print("=" * 50)

start_time = time.time()
generated_images = []
metadata = []

for i, (prompt, seed) in enumerate(zip(DATASET_PROMPTS, SEEDS), 1):
    print(f"\n🖼️ Gerando imagem {i}/{NUM_IMAGES} (seed: {seed})")
    print(f"🎭 Variação: {['Frontal Studio', 'Three-quarter Natural', 'Profile Artistic', 'Looking Up Golden', 'Close-up Fashion', 'Reference Frontal'][i-1]}")
    
    try:
        # Geração com parâmetros premium
        image = pipeline(
            prompt=prompt,
            num_inference_steps=INFERENCE_STEPS,
            guidance_scale=GUIDANCE_SCALE,
            width=RESOLUTION,
            height=RESOLUTION,
            generator=torch.Generator(device="cuda").manual_seed(seed)
        ).images[0]
        
        # Salvar imagem
        filename = f"valentina_dataset_{i:02d}.png"
        filepath = f"{COLAB_DATASET_PATH}/{filename}"
        image.save(filepath)
        
        # Metadados
        meta = {
            "image": filename,
            "prompt": prompt,
            "seed": seed,
            "steps": INFERENCE_STEPS,
            "guidance": GUIDANCE_SCALE,
            "resolution": f"{RESOLUTION}x{RESOLUTION}",
            "base_model": BASE_MODEL_ID,
            "nsfw_lora": NSFW_LORA_ID,
            "nsfw_weight": NSFW_LORA_WEIGHT,
            "midjourney_lora": USE_MIDJOURNEY_LORA,
            "midjourney_weight": MIDJOURNEY_LORA_WEIGHT if USE_MIDJOURNEY_LORA else None
        }
        
        generated_images.append(image)
        metadata.append(meta)
        
        print(f"✅ Imagem {i} concluída: {filepath}")
        
        # Limpeza de memória a cada imagem
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"❌ Erro na imagem {i}: {e}")
        continue

# Tempo total
end_time = time.time()
total_time = end_time - start_time
print(f"\n⏱️ Tempo total: {total_time:.1f}s ({total_time/60:.1f}min)")
print(f"🏆 Dataset premium concluído: {len(generated_images)}/{NUM_IMAGES} imagens")

# Salvar metadados
metadata_path = f"{COLAB_DATASET_PATH}/dataset_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"📊 Metadados salvos: {metadata_path}")

In [ ]:
# 📦 CRIAÇÃO DO PACOTE FINAL PARA DOWNLOAD
import zipfile
from google.colab import files
import os

print("📦 Criando pacote do dataset premium...")

# Criar README para o dataset
readme_content = f"""# Valentina High-Quality Dataset

## Informações da Geração
- **Modelo Base**: {BASE_MODEL_ID} (máxima qualidade)
- **NSFW LoRA**: {NSFW_LORA_ID} (peso: {NSFW_LORA_WEIGHT})
- **Midjourney LoRA**: {'Sim' if USE_MIDJOURNEY_LORA else 'Não'}{f' (peso: {MIDJOURNEY_LORA_WEIGHT})' if USE_MIDJOURNEY_LORA else ''}
- **Arquitetura**: Sequential LoRA (Base + NSFW + {f'Midjourney' if USE_MIDJOURNEY_LORA else 'N/A'})
- **GPU**: {GPU_NAME} com {INFERENCE_STEPS} steps
- **Resolução**: {RESOLUTION}x{RESOLUTION}
- **Guidance Scale**: {GUIDANCE_SCALE}
- **Total de Imagens**: {len(generated_images)}

## Arquitetura Sequential LoRA
1. **Base Model**: FLUX.1-dev (modelo oficial Black Forest Labs)
2. **1º LoRA**: NSFW_MASTER_FLUX (HuggingFace automático)
3. **2º LoRA**: Midjourney LoRA {'(aplicada)' if USE_MIDJOURNEY_LORA else '(não aplicada)'}

## Variações Incluídas
1. **Frontal Studio**: Headshot profissional, fundo branco
2. **Three-quarter Natural**: Ângulo 3/4, luz natural
3. **Profile Artistic**: Perfil dramático, rim lighting
4. **Looking Up Golden**: Perspectiva inferior, golden hour
5. **Close-up Fashion**: Close-up intenso, fashion style
6. **Reference Frontal**: Referência frontal limpa

## Qualidade Premium
- Gerado no Google Colab com {GPU_NAME}
- {INFERENCE_STEPS} steps de inferência para máxima qualidade
- FLUX.1-dev base model + Sequential LoRAs
- Controle independente de pesos dos LoRAs
- Múltiplas poses e iluminações para dataset diverso

## Uso Recomendado
- **Treinamento LoRA**: Substitua o dataset original por este
- **Trigger Word**: vltna woman
- **Qualidade**: Superior para treinamento de alta fidelidade
- **Vantagem**: Base model FLUX.1-dev + LoRAs sequenciais = qualidade máxima

---
Gerado em: {time.strftime('%Y-%m-%d %H:%M:%S UTC')}
Tempo de geração: {total_time:.2f} segundos
"""

readme_path = f"{COLAB_OUTPUT_PATH}/README.md"
with open(readme_path, 'w') as f:
    f.write(readme_content)

# Criar ZIP com tudo
zip_filename = "valentina_hq_dataset_flux_sequential.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, compresslevel=6) as zipf:
    # Adicionar todas as imagens
    for i, img_file in enumerate([f"valentina_dataset_{j:02d}.png" for j in range(1, len(generated_images)+1)]):
        img_path = f"{COLAB_DATASET_PATH}/{img_file}"
        if os.path.exists(img_path):
            zipf.write(img_path, f"images/{img_file}")
    
    # Adicionar metadados
    zipf.write(metadata_path, "metadata/dataset_metadata.json")
    
    # Adicionar README
    zipf.write(readme_path, "README.md")

# Verificar tamanho final
zip_size_mb = os.path.getsize(zip_filename) / (1024*1024)
print(f"✅ Pacote criado: {zip_filename} ({zip_size_mb:.1f}MB)")

# Listar conteúdo
print(f"\n📋 Conteúdo do dataset:")
print(f"   • {len(generated_images)} imagens PNG em alta resolução")
print(f"   • Metadados JSON com configurações")
print(f"   • README com documentação completa")

print(f"\n🎯 RESUMO DO DATASET PREMIUM:")
print(f"📊 {len(generated_images)} imagens em qualidade máxima")
print(f"⚡ {INFERENCE_STEPS} steps com {GPU_NAME}")
print(f"🎨 FLUX.1-dev + Sequential LoRAs (NSFW + {'Midjourney' if USE_MIDJOURNEY_LORA else 'N/A'})")
print(f"📀 Resolução {RESOLUTION}x{RESOLUTION}")
print(f"🎭 6 variações de pose e iluminação")
print(f"🔥 Qualidade superior ao dataset MacBook M3")

print(f"\n📥 Baixando dataset premium...")
files.download(zip_filename)

print(f"\n🎉 DATASET DE ALTA QUALIDADE PRONTO!")
print(f"💡 Substitua o dataset original por este para melhor qualidade de treinamento")
print(f"🚀 Use este dataset no notebook de treinamento para resultados superiores")

In [ ]:
# 🧹 LIMPEZA OPCIONAL (DESCOMENTE PARA LIBERAR MEMÓRIA)
# print("🧹 Limpando memória...")
# del pipeline
# torch.cuda.empty_cache()
# print("✅ Memória liberada")

print("\n🎯 GERAÇÃO COMPLETA!")
print("📋 Próximos passos:")
print("1. Baixe o arquivo valentina_hq_dataset.zip")
print("2. Substitua o dataset original por este")
print("3. Execute o notebook de treinamento LoRA")
print("4. Aproveite a qualidade superior!")